In [34]:
import os
from ncbi.names import convert_expected, generate_names_df
import pandas as pd
from typing import List
import re

In [35]:
def underscore_split(exp_df: pd.DataFrame) -> List[List[str]]:
    exp_names = exp_df.index.tolist()

    # print(exp_names)
    # Replace any instance of sp with sp.
    exp_names = [re.sub("_sp_", "_sp._", x) for x in exp_names]

    # There is also a case where sp is at the end, so replace that too.
    exp_names = [re.sub("_sp$", "_sp.", x) for x in exp_names]

    # print(exp_names)

    exp_split_names = [x.split("_") for x in exp_names]

    return exp_split_names

def whitespace_split(exp_df: pd.DataFrame) -> List[List[str]]:
    exp_names = exp_df.index.tolist()
    exp_split_names = [x.split() for x in exp_names]

    return exp_split_names

split_dict = {
    "hilo": underscore_split,
    "mixed": underscore_split,
    "bmock12": whitespace_split,
    "camisimGI": whitespace_split,
    "nist": underscore_split,
    "tourlousse": underscore_split
}

# replacement_dict = {
#     "Treponema caldarium": "Gracilinema caldarium",
#     "Pseudomonas stutzeri": "Stutzerimonas stutzeri",
# }

In [36]:
# Find everything in target directory with "expected_species_annotated" in the name.
target_dir = "../pipelines_old"
def find_expected(dir: str):
    for root, dirs, files in os.walk(dir):
        for file in files:
            if file.endswith("expected_species_annotated.csv"):
                yield os.path.join(root, file)

def drop_id_and_reannotate(f):
    new_path = f.replace("pipelines_old", "expected_pipelines")
    new_path = new_path.replace("expected_species_annotated", "expected_species")

    print(new_path)

    new_df = pd.read_csv(f, sep=",", index_col=0)

    # Drop tax_id column.
    new_df.drop(new_df.columns[1], axis=1, inplace=True)

    # Save to new path.
    # new_df.to_csv(new_path, index_label="Species")

    source = new_path.split("/")[-2]
    print(source)

    convert_expected(new_path, split_func=split_dict[source])

def copy_to_expected_dir(f: str):
    new_path = f.replace("pipelines_old", "expected_pipelines")

    new_df = pd.read_csv(f, sep=",", index_col=0)

    # Save to new path.
    new_df.to_csv(new_path, index_label="Species")

for f in find_expected(target_dir):
    # Now, we are going to reannotate the files.
    drop_id_and_reannotate(f)

../expected_pipelines/amos/mixed/expected_species.csv
mixed
The pkl file was last modified (and hopefully generated) on 2023-02-14 18:37:13.183117+00:00
                                  RA                    split_name
Species                                                           
AKKERMANSIA_MUCINIPHILA       0.0637       AKKERMANSIA_MUCINIPHILA
ALISTIPES_FINEGOLDII          0.0454          ALISTIPES_FINEGOLDII
ANAEROSTIPES_HADRUS           0.0611           ANAEROSTIPES_HADRUS
BACTEROIDES_THETAIOTAOMICRON  0.0269  BACTEROIDES_THETAIOTAOMICRON
BACTEROIDES_UNIFORMIS         0.0366         BACTEROIDES_UNIFORMIS
BIFIDOBACTERIUM_LONGUM        0.1292        BIFIDOBACTERIUM_LONGUM
BLAUTIA_WEXLERAE              0.0377              BLAUTIA_WEXLERAE
CLOSTRIDIUM_BUTYRICUM         0.0370         CLOSTRIDIUM_BUTYRICUM
COLLINSELLA_AEROFACIENS       0.0695       COLLINSELLA_AEROFACIENS
ESCHERICHIA_COLI              0.0326              ESCHERICHIA_COLI
EUBACTERIUM_HALLII            0.0516       

In [37]:
# We need to rerun the annotater on the new files because the NCBI taxonomy has changed or somehave been merged.
